# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Importing all the Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
file="D:\Homework\API-Challenge\VacationPy\Output Data/Cities.csv"

city_df = pd.read_csv(file)
city_df.head()



,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
#COnfiguring gmaps
gmaps.configure(api_key=g_key)

# Using the Lat and Lng as locations and Humidity as the weight
locations= city_df[["Lat","Lng"]]
weights=city_df["Humidity"]

#Setting the max_intensity to the maximum humidity value
maxhumidity= city_df["Humidity"].max()


#PLotting the heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Creating and adding the heat layer on the map with max_humidity as max_intensity and point)radius =3 for a better visulaisation
heat_layer = gmaps.heatmap_layer(locations, weights= weights, 
                                 dissipating=False, max_intensity=maxhumidity,point_radius=3)

fig.add_layer(heat_layer)

# Displaing figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Creating an ideal dataframe with the cities with idal temperature, wind speed, humidity
ideal_df = city_df.loc[(city_df["Max Temp"]<80) & (city_df["Max Temp"]>70) &
                      (city_df["Cloudiness"]==0) &
                      (city_df["Wind Speed"] <=10) , :]


#Cleaning the ideal_df for better visualization
ideal_df= ideal_df.dropna(how='any')
ideal_df.reset_index(inplace =True)
del ideal_df["index"]

#Displaying the ideal dataframe
ideal_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
5,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
6,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#Creating a hotel dataframe with an new column "Hotel Name"
hotel_df= ideal_df.loc[:,["City","Country","Lat","Lng"]]
hotel_df["Hotel Name"] =""

#Displaying the Hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,ponta do sol,BR,-20.63,-46.00,
1,cidreira,BR,-30.17,-50.22,
2,morondava,MG,-20.30,44.28,
3,vaini,IN,15.34,74.49,
4,nador,MA,35.17,-2.93,
5,mogok,MM,22.92,96.51,
6,birjand,IR,32.86,59.22,


In [ ]:

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
        "keyword": "hotel",
        "radius": 5000,
        "type": "hotel",
        "key": g_key
            }

for index, row in hotel_df.iterrows():
    # getting  city name, lat, lnt from  hotel_df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assembling url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extracting results
    results = response['results']
    
    # saving the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, showing missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Waiting for 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Printing end of search once searching is completed
print("-------End of Search-------")


In [9]:
#Displaying the whole dataframe with the Hotel Name
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,ponta do sol,BR,-20.63,-46.00,
1,cidreira,BR,-30.17,-50.22,
2,morondava,MG,-20.30,44.28,
3,vaini,IN,15.34,74.49,
4,nador,MA,35.17,-2.93,
5,mogok,MM,22.92,96.51,
6,birjand,IR,32.86,59.22,


In [10]:
# NOTE: Do not change any of the code in this cell


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Adding the  marker layer on top of the heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Displaying Map and Figure
fig

Figure(layout=FigureLayout(height='420px'))